# Manual Computation #

## Prerequisites ##

In [0]:
!pip install rasterio

In [0]:
import PIL.Image
import rasterio as rio
import numpy as np

## Mount (colab only) ##

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Load and Display Data ##

In [0]:
with rio.open('/content/gdrive/My Drive/berlin-imagery.tif', 'r') as ds:
  imagery_data = ds.read()

In [0]:
with rio.open('/content/gdrive/My Drive/berlin-ndwi.tif', 'r') as ds:
  prediction_data = ds.read()

In [0]:
pred_lo = np.min(prediction_data)
pred_hi = np.max(prediction_data)
prediction_grayscale = (prediction_data - pred_lo) / (pred_hi - pred_lo)
prediction_grayscale = (prediction_grayscale * 0xff)[0].astype(np.uint8)

In [0]:
PIL.Image.fromarray(prediction_grayscale)

In [0]:
μ0, σ0, μ1, σ1 =  -0.3616522253, 0.1555788219, 0.3314088285, 0.1071685851

In [0]:
midpoint = (μ0*σ1 + μ1*σ0)/(σ0 + σ1)

In [0]:
prediction_final = ((prediction_data > midpoint) * 0xff)[0].astype(np.uint8)

In [0]:
PIL.Image.fromarray(prediction_final)

## Compute Percentage ##

In [0]:
imagery_nodata = 0

In [0]:
valid_mask = (imagery_data[0] != imagery_nodata)

In [0]:
foreground_count = ((prediction_final > 0) * valid_mask).sum()
foreground_count

In [0]:
background_count = ((prediction_final <= 0) * valid_mask).sum()
background_count

In [0]:
float(foreground_count) / (background_count + foreground_count)

# Batch Computation #

### Step 1 ###

Clone the [`geotrellis/deeplab-nlcd`](https://github.com/geotrellis/deeplab-nlcd) repository to a local directory.

Type
```
git clone git@github.com:geotrellis/deeplab-nlcd.git
```
or similar.

### Step 2 ###

Enter the root of the repository directory.

Type
```
cd deeplab-nlcd
```
or similar.

### Step 3 ###

Start a docker container with the needed dependencies.

Type
```
docker run -it --rm -w /workdir -v $(pwd):/workdir -v $HOME/Desktop:/desktop --runtime=nvidia jamesmcclain/aws-batch-ml:9 bash
```
or similar.  This sample command line will mount the local directory `~/Desktop/` which is assumed to contain the imagery on which we wish to work.  We will see later that it is also possible to use imagery on S3.

### Step 4 ###

Now calculate the spectral discrimination index using imagery, predictions, and "ground truth".

Type
```
python /workdir/python/local/percentage.py --prediction /desktop/predictions/image* --prediction-nd 255 --imagery /desktop/imagery/image* --imagery-nd 0 --data "-0.3616522253 0.1555788219 0.3314088285 0.1071685851"
```
or similar.

Note that `~/Desktop/imagery/` is assumed to contain the imagery (files with names matching the pattern `image*.tif`) and the directory `~/Desktop/predictions/` is assumed to contain the predictions (files with names matching the pattern `image*.tif`).

The four numbers passed in a single string argument to `--data` are μ0, σ0, μ1, and σ1, respectively.